In [1]:
import torch

from model.resnet_model import CNNClassifier
from torch.utils.data import DataLoader
from pathlib import Path
from ml_utils import CustomDataset

from sklearn.metrics import f1_score, accuracy_score
from torch.nn import CrossEntropyLoss

from tqdm import tqdm

In [2]:
device = "mps"

In [3]:
test_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="test")
test_loader = DataLoader(test_dt, batch_size=64, shuffle=True, num_workers=4)

cls = CNNClassifier(num_classes=196, pretrained=True, model_name="resnet18", freeze_layers=0)
cls.load_state_dict(torch.load("../experiments/results/r18_pretrained_fre0.pth", weights_only=True))
cls.eval()
cls = cls.to(device)

In [4]:

losses, preds, trues = [], [], []
criterion = CrossEntropyLoss().to(device)

with torch.no_grad():  # Disable gradient calculations for inference
    for idx, (x, y) in enumerate(tqdm(test_loader), 1):
        if device != "cpu":
            x, y = x.to(device), y.to(device)  # Move data to the specified device
        logits = cls(x)  # Forward pass
        loss = criterion(logits, y)  # Calculate loss
        predictions = torch.argmax(logits, dim=1)  # Get class predictions

        losses.append(loss)
        preds.append(predictions)
        trues.append(y)

    # Concatenate all predictions, true labels, and losses
    preds = torch.cat(preds).cpu().numpy()
    trues = torch.cat(trues).cpu().numpy()
    losses_tensor = torch.stack(losses)

# Calculate and store metrics
loss_mean = losses_tensor.mean().item()
loss_std = losses_tensor.std().item()
f1_mean = f1_score(trues, preds, average='weighted')
acc_mean = accuracy_score(trues, preds)

100%|██████████| 126/126 [01:11<00:00,  1.77it/s]


In [5]:
loss_mean

0.5139766335487366

In [6]:
f1_mean

0.8835541083535506

In [8]:
acc_mean

0.8838452928740207